Nog een keer hetzelfde programma alleen deze keer maak ik het met CustomTkinter. Dat is een Python UI library based on Tkinter die een nieuw, modernere widgets heeft.

In [2]:
import sys
!{sys.executable} -m pip install customtkinter

     ------------------------------------- 294.0/294.0 kB 17.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import tkinter
import customtkinter

customtkinter.set_appearance_mode("Dark")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("green")  # Themes: blue (default), dark-blue, green

app = customtkinter.CTk()  # create CTk window like you do with the Tk window
app.geometry("400x240")

def button_function():
    print("button pressed")


# Use CTkButton instead of tkinter Button
button = customtkinter.CTkButton(master=app, text="CTkButton", command=button_function)
button.place(relx=0.5, rely=0.5, anchor=tkinter.CENTER)

app.mainloop()

CustomTkinter can adapt to the Windows 10/11 light or dark mode